In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
import torch
import numpy as np
import pandas as pd

from transformers import AutoTokenizer

In [57]:
df = pd.DataFrame(
    [
        [0.3, 1.2, 0.05, 1.07],
        [4.18, 3.2, 0.09, 0.05],
        [0.85, 0.27, 2.2, 5.03],
        [0.23, 0.57, 6.12, 5.1]
    ]
)
data = torch.tensor(df.values, dtype=torch.float32)

In [58]:
df.values

array([[0.3 , 1.2 , 0.05, 1.07],
       [4.18, 3.2 , 0.09, 0.05],
       [0.85, 0.27, 2.2 , 5.03],
       [0.23, 0.57, 6.12, 5.1 ]])

In [59]:
def contrastive_loss(data):
    target = torch.arange(data.size(0))
    loss = torch.nn.CrossEntropyLoss()(data, target)
    return loss

In [52]:
N = data.size(0)
non_diag_mask = ~torch.eye(N, N, dtype=bool)

In [47]:
data[non_diag_mask]

tensor([1.2000, 0.0500, 1.0700, 0.1800, 0.0900, 0.0500, 0.8500, 0.2700, 1.0300,
        0.2300, 0.5700, 0.1200])

In [55]:
data = torch.tensor(df.values, dtype=torch.float32)
data[range(N), range(N)] += 0.5
data[non_diag_mask] -= 0.02
print(data)
print(f"Loss = {contrastive_loss(data)}")

tensor([[0.8000, 1.1800, 0.0300, 1.0500],
        [4.1600, 3.7000, 0.0700, 0.0300],
        [0.8300, 0.2500, 2.7000, 5.0100],
        [0.2100, 0.5500, 6.1000, 5.6000]])
Loss = 1.4527289867401123


In [60]:
torch.nn.Softmax(dim=1)(data)

tensor([[0.1563, 0.3844, 0.1217, 0.3376],
        [0.7102, 0.2665, 0.0119, 0.0114],
        [0.0141, 0.0079, 0.0545, 0.9235],
        [0.0020, 0.0028, 0.7314, 0.2637]])

In [61]:
torch.nn.Softmax(dim=1)(data).sum(dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000])

In [69]:
N = data.size(0)
non_diag_mask = ~torch.eye(N, N, dtype=bool)

for inx in range(5):
    data = torch.tensor(df.values, dtype=torch.float32)
    data[range(N), range(N)] += inx*0.5
    data[non_diag_mask] -= inx*0.02
    print(data)
    print(f"Loss = {contrastive_loss(data)}")


tensor([[0.3000, 1.2000, 0.0500, 1.0700],
        [4.1800, 3.2000, 0.0900, 0.0500],
        [0.8500, 0.2700, 2.2000, 5.0300],
        [0.2300, 0.5700, 6.1200, 5.1000]])
Loss = 1.8551747798919678
tensor([[0.8000, 1.1800, 0.0300, 1.0500],
        [4.1600, 3.7000, 0.0700, 0.0300],
        [0.8300, 0.2500, 2.7000, 5.0100],
        [0.2100, 0.5500, 6.1000, 5.6000]])
Loss = 1.4527289867401123
tensor([[1.3000, 1.1600, 0.0100, 1.0300],
        [4.1400, 4.2000, 0.0500, 0.0100],
        [0.8100, 0.2300, 3.2000, 4.9900],
        [0.1900, 0.5300, 6.0800, 6.1000]])
Loss = 1.0994977951049805
tensor([[ 1.8000,  1.1400, -0.0100,  1.0100],
        [ 4.1200,  4.7000,  0.0300, -0.0100],
        [ 0.7900,  0.2100,  3.7000,  4.9700],
        [ 0.1700,  0.5100,  6.0600,  6.6000]])
Loss = 0.8030187487602234
tensor([[ 2.3000,  1.1200, -0.0300,  0.9900],
        [ 4.1000,  5.2000,  0.0100, -0.0300],
        [ 0.7700,  0.1900,  4.2000,  4.9500],
        [ 0.1500,  0.4900,  6.0400,  7.1000]])
Loss = 0.5657716989

In [71]:
torch.nn.Softmax(dim=1)(data).sum(dim=1)

tensor([1., 1., 1., 1.])

In [72]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, output_embed_dim):
        super().__init__()
        self.embedding_layer = torch.nn.Embedding(vocab_size, embed_dim)
        self.encoder = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(embed_dim, nhead=8, batch_first=True),
            num_layers=3,
            norm=torch.nn.LayerNorm([embed_dim]),
            enable_nested_tensor=False
        )
        self.projection = torch.nn.Linear(embed_dim, output_embed_dim)
    
    def forward(self, tokenizer_output):
        x = self.embedding_layer(tokenizer_output['input_ids'])
        x = self.encoder(x, src_key_padding_mask=tokenizer_output['attention_mask'].logical_not())
        cls_embed = x[:,0,:]
        return self.projection(cls_embed)

In [73]:
def train_loop(dataset):
    embed_size = 512
    output_embed_size = 128
    max_seq_len = 64
    batch_size = 32

    # define the question/answer encoders
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    question_encoder = Encoder(tokenizer.vocab_size, embed_size, 
                               output_embed_size)
    answer_encoder = Encoder(tokenizer.vocab_size, embed_size, 
                             output_embed_size)

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                             shuffle=True)    
    optimizer = torch.optim.Adam(
        list(question_encoder.parameters()) + list(answer_encoder.parameters()
    ), lr=1e-5)
    loss_fn = torch.nn.CrossEntropyLoss()

    running_loss = []
    for _, data_batch in enumerate(dataloader):

        # Tokenize the question/answer pairs (each is a batch of 32 questions and 32 answers)
        question, answer = data_batch
        question_tok = tokenizer(question, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)
        answer_tok = tokenizer(answer, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)

        # Compute the embeddings: the output is of dim = 32 x 128
        question_embed = question_encoder(question_tok)
        answer_embed = answer_encoder(answer_tok)

        # Compute similarity scores: a 32x32 matrix
        # row[N] reflects similarity between question[N] and answers[0...31]
        similarity_scores = question_embed @ answer_embed.T

        # we want to maximize the values in the diagonal
        target = torch.arange(question_embed.shape[0], dtype=torch.long)
        loss = loss_fn(similarity_scores, target)
        running_loss += [loss.item()]

        # this is where the magic happens
        optimizer.zero_grad()    # reset optimizer so gradients are all-zero
        loss.backward()
        optimizer.step()

    return question_encoder, answer_encoder

In [96]:
def train(dataset, num_epochs=1):
    embed_size = 512
    output_embed_size = 128
    max_seq_len = 64
    batch_size = 32

    n_iters = len(dataset) // batch_size + 1
    
    # define the question/answer encoders
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    question_encoder = Encoder(tokenizer.vocab_size, embed_size, output_embed_size)
    answer_encoder = Encoder(tokenizer.vocab_size, embed_size, output_embed_size)

    # define the dataloader, optimizer and loss function    
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)    
    optimizer = torch.optim.Adam(list(question_encoder.parameters()) + list(answer_encoder.parameters()), lr=1e-5)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        running_loss = []
        for idx, data_batch in enumerate(dataloader):

            # Tokenize the question/answer pairs (each is a batc of 32 questions and 32 answers)
            question, answer = data_batch
            question_tok = tokenizer(question, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)
            answer_tok = tokenizer(answer, padding=True, truncation=True, return_tensors='pt', max_length=max_seq_len)
            if inx == 0 and epoch == 0:
                print(question_tok['input_ids'].shape, answer_tok['input_ids'].shape)
            
            # Compute the embeddings: the output is of dim = 32 x 128
            question_embed = question_encoder(question_tok)
            answer_embed = answer_encoder(answer_tok)
            if inx == 0 and epoch == 0:
                print(question_embed.shape, answer_embed.shape)
    
            # Compute similarity scores: a 32x32 matrix
            # row[N] reflects similarity between question[N] and answers[0...31]
            similarity_scores = question_embed @ answer_embed.T
            if inx == 0 and epoch == 0:
                print(similarity_scores.shape)
    
            # we want to maximize the values in the diagonal
            target = torch.arange(question_embed.shape[0], dtype=torch.long)
            loss = loss_fn(similarity_scores, target)
            running_loss += [loss.item()]
            if idx == n_iters-1:
                print(f"Epoch {epoch}, loss = ", np.mean(running_loss))
    
            # this is where the magic happens
            optimizer.zero_grad()    # reset optimizer so gradients are all-zero
            loss.backward()
            optimizer.step()

    return question_encoder, answer_encoder

In [97]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, datapath):
        self.data = pd.read_csv(datapath, sep="\t", nrows=300)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data.iloc[idx]['questions'], self.data.iloc[idx]['answers']

dataset = MyDataset('qa_dataset.tsv')
dataset.data.head(5)

,questions,answers
0,who played bubba in the tv series in the heat ...,Carlos Alan Autry Jr. (also known for a period...
1,where did the 2017 tour de france start,"The 3,540 km (2,200 mi)-long race commenced wi..."
2,who is the chess champion of the world,Current world champion Magnus Carlsen won the ...
3,who scored the most hat tricks in football,Cristiano Ronaldo and Messi have scored three ...
4,what do you need to be an ontario scholar,Ontario Scholars are high school graduates in ...


In [ ]:
qe, ae = train(dataset, num_epochs=5)

In [ ]:
question = 'What is the tallest mountain in the world?'
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
question_tok = tokenizer(question, padding=True, truncation=True, return_tensors='pt', max_length=64)
question_emb = qe(question_tok)[0]
print(question_tok)
print(question_emb[:5])


{'input_ids': tensor([[  101,  2054,  2003,  1996, 13747,  3137,  1999,  1996,  2088,  1029,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([-0.1229,  0.4092,  0.4142, -0.0284,  0.0784], grad_fn=<SliceBackward0>)


In [ ]:
answers = [
    "What is the tallest mountain in the world?",
    "The tallest mountain in the world is Mount Everest.",
    "Who is donald duck?"
]
answer_tok = []
answer_emb = []
for answer in answers:
    tok = tokenizer(answer, padding=True, truncation=True, return_tensors='pt', max_length=64)
    answer_tok.append(tok['input_ids'])
    emb = ae(tok)[0]
    answer_emb.append(emb)

print(answer_tok)
print(answer_emb[0][:5])
print(answer_emb[1][:5])
print(answer_emb[2][:5])

[tensor([[  101,  2054,  2003,  1996, 13747,  3137,  1999,  1996,  2088,  1029,
           102]]), tensor([[  101,  1996, 13747,  3137,  1999,  1996,  2088,  2003,  4057, 23914,
          1012,   102]]), tensor([[ 101, 2040, 2003, 6221, 9457, 1029,  102]])]
tensor([-0.5402,  0.8633, -0.2744, -0.2967, -0.2044], grad_fn=<SliceBackward0>)
tensor([-0.8422,  0.7184, -0.1301, -0.4388, -0.2017], grad_fn=<SliceBackward0>)
tensor([-0.6134,  0.3070,  0.1103, -0.3892, -0.5599], grad_fn=<SliceBackward0>)


In [99]:
question_emb @ torch.stack(answer_emb).T

tensor([1.0004, 1.0827, 0.0563], grad_fn=<SqueezeBackward4>)